In [22]:
import wget

In [24]:
#Docoment.json download
# url="https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"
# wget.download(url)

In [4]:
import json

In [27]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [28]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


#1 Question


Pull the latest elastic search

docker run -it --rm --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" elasticsearch:8.17.6


after run this :

curl http://localhost:9200
and get the hashsvalue.

In [134]:
import json
from elasticsearch import Elasticsearch

In [135]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [136]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


In [137]:
from elasticsearch import Elasticsearch

es_client  = Elasticsearch("http://localhost:9200")
print(es_client.info())
#dbcbbbd0bc4924cfeb28929dc05d82d662c527b7

{'name': 'bfb15c857c78', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Gy9E7mphRpOOUlgzXoPA-g', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Index Settings

In [ ]:
# if es_client.indices.exists(index=index_name):
#     es_client.indices.delete(index=index_name)
#     print(f"Deleted index: {index_name}")


Deleted index: course-questions


In [139]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name="course-questions"
es_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#2 Question

In [140]:
for doc in documents:
    es_client.index(index=index_name,document=doc)

#3 Question

In [141]:
def elastic_search (query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    response=es_client.search(index=index_name,body=search_query)

    results = []
    for hit in response['hits']['hits']:
        results.append({
            'score': hit['_score'],
            'source': hit['_source']
        })
    return results

In [142]:
query="How do execute a command on a Kubernetes pod"

In [143]:
elastic_search(query)

[{'score': 44.50556,
  'source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'score': 35.433445,
  'source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'score': 33.70974,
  'source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to co

#4 Question

In [145]:
def elastic_search (query):
    search_query={
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response=es_client.search(index=index_name,body=search_query)

    results = []
    for hit in response['hits']['hits']:
        results.append({
            'score': hit['_score'],
            'source': hit['_source']
        })
    return results


In [146]:
query="How do copy a file to a Docker container?"

In [147]:
elastic_search(query)

[{'score': 73.38676,
  'source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'score': 66.688705,
  'source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'questio

In [148]:
search_results = elastic_search(query)


third_question = results[2]['source']['question']
print(third_question)

How do I copy files from a different folder into docker container’s working directory?


In [149]:
from openai import OpenAI

In [150]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)


#5 Question

In [151]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_entries = [
        context_template.format(
            question=doc['question'],
            text=doc['text']
        ) for doc in search_results
    ]

    context = "\n\n".join(context_entries)

    prompt = prompt_template.format(
        question=query,
        context=context
    )

    return prompt


In [ ]:
query = "How do I execute a command in a running docker container?"


results = elastic_search(query)

# Flatten for prompt building
flat_results = [doc['source'] for doc in results]


prompt = build_prompt(query, flat_results)

print(prompt)
print("Prompt length:", len(prompt))



You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

#6 Question

In [ ]:
import tiktoken


encoding = tiktoken.encoding_for_model("gpt-4o")


prompt = build_prompt(query, flat_results)

tokens = encoding.encode(prompt)


num_tokens = len(tokens)

print("Number of tokens in prompt:", num_tokens)


Number of tokens in prompt: 322
